In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Convert categorical columns to numerical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode 'sex', 'smoker', and 'region'
le = LabelEncoder()
dataset['sex'] = le.fit_transform(dataset['sex'])       # male:1, female:0
dataset['smoker'] = le.fit_transform(dataset['smoker']) # yes:1, no:0
dataset['region'] = le.fit_transform(dataset['region']) # region names to 0-3

# Split the dataset into training and test sets (80% train, 20% test)
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=0)

# Separate the labels (expenses) from the features
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

# Normalize the features (optional but improves performance)
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

# Build the model
def build_model():
  model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss='mse',
                metrics=['mae', 'mse'])
  return model

model = build_model()

# Train the model
EPOCHS = 100
history = model.fit(normed_train_data, train_labels,
                    epochs=EPOCHS, validation_split=0.2, verbose=0)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
